## Изучение общей информации о данных

Прочитаем файл и сохраним данные в переменной:

In [6]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строк таблицы:

In [7]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Основная информация о данных:

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Подсчитаем общее количество пропущенных значений:

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски в столбце с доходами медианным значением в зависимости от типа занятости:

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Заменим все отрицательные значения положительными в столбце с количеством дней трудового стажа:

In [11]:
data['days_employed'] = data['days_employed'].abs()

Выведем медианное значение трудового стажа по каждому типу занятости:

In [12]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Однако этот столбец больше не понадобится для исследования, так что можно не исправлять эти значения.

Выведем перечень уникальных значений столбца `children`:

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Удалим аномальные значения из столбца `children`:

In [14]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем перечень уникальных значений столбца, чтобы убедиться, что артефакты удалены:

In [15]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости:

In [16]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что пропусков больше нет:

In [17]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный:

In [18]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Приведем все значения в столбце `education` к нижнему регистру:

In [19]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных и затем удалим их:

In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()

### Категоризация данных

На основании данных диапазонов, создадим в датафрейме столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [23]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`:

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Напишем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [25]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [26]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Поиск зависимости между количеством детей и возвратом кредита в срок.

In [27]:
data_filtered_debt = data[data['debt'] != 0]
data_filtered_debt = data_filtered_debt.groupby('children')['debt'].count()
data_filtered_all = data.groupby('children')['debt'].count()
chance = (data_filtered_debt / data_filtered_all)*100

In [28]:
value = {'clients_with_children': data_filtered_all,'clients_with_debt': data_filtered_debt,'debt_chance': chance}
represent = pd.DataFrame(data=value)
represent = represent.fillna(0)
display(represent.sort_values(by='debt_chance', ascending=False))

,clients_with_children,clients_with_debt,debt_chance
children,,,
4,41,4.0,9.756098
2,2052,194.0,9.454191
1,4808,444.0,9.234609
3,330,27.0,8.181818
0,14091,1063.0,7.543822
5,9,0.0,0.000000


**Вывод:** Зависимость между количеством детей и возвратом кредита в срок прослеживается: Чем больше детей у клиента, тем больше вероятность того, что он вернет кредит в срок. Вероятность задолженности среди клиентов без детей составляет 7.5%, это самый маленький показатель среди групп, так как это самая многочисленная группа. Вероятность задолженности клиентов с 1 и 2 детьми равным 9.2% и 9.4% соответсвенно, у клиентов с 3 детьми вероятность составляет ~ 8.2% и у клиентов с 4 детьми вероятность задолженности равна 9.7%. Поскольку задолженников среди клиентов с 5 детьми нет, то вероятность невозврата клиентов данной группы равна 0.

### Поиск зависимости между семейным положением и возвратом кредита в срок.

In [29]:
family_debt = data[data['debt'] != 0]
family_debt = family_debt.groupby('family_status')['debt'].count()
all_statuses = data.groupby('family_status')['debt'].count()
chance_status = (family_debt / all_statuses)*100


In [30]:
value = {'clients_overall': all_statuses,'clients_with_debt': family_debt,'debt_chance': chance_status}
represent = pd.DataFrame(data=value)
display(represent.sort_values(by='debt_chance', ascending=False))

,clients_overall,clients_with_debt,debt_chance
family_status,,,
Не женат / не замужем,2796,273,9.763948
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606


**Вывод:** Вероятность того, что клиент, не состоящий в официальном браке, не вернет кредит в срок составляет ~ 9.8%. Это выше вероятности невозврата клиентов, состоящих в браке ~ 7.6%. Самый маленький процент невозврата наблюдается вдов и вдовцов, он равен 6.6%. Вероятность образования задолженности среди клиентов в разводе и состоящих в гражданском браке составляет 7% и 9.3% соответсвенно.

### Поиск зависимости между уровнем дохода и возвратом кредита в срок.

In [31]:
category_debt = data[data['debt'] != 0]
category_debt = category_debt.groupby('total_income_category')['debt'].count()
overall_category = data.groupby('total_income_category')['debt'].count()
chance_status = (category_debt / overall_category)*100

In [32]:
value = {'clients_overall': overall_category,'clients_with_debt': category_debt,'debt_chance': chance_status, 'income': [ '1000001 и выше', '200001 – 1000000','50001 – 200000', '30001 - 50000',  '0 - 30000']}
represent = pd.DataFrame(data=value)
display(represent.sort_values(by='debt_chance', ascending=False))

,clients_overall,clients_with_debt,debt_chance,income
total_income_category,,,,
E,22,2,9.090909,0 - 30000
C,15921,1353,8.498210,50001 – 200000
A,25,2,8.000000,1000001 и выше
B,5014,354,7.060231,200001 – 1000000
D,349,21,6.017192,30001 - 50000


**Вывод:** Большинство должников относятся к категории "С" с доходом от 50001 до 200000, меньше всего должников находится в категориях "А" и "Е". Наибольшая вероятность невозврата наблюдается в категории "Е" - 9%. Можно предположить, что это происходит из-за недостатка средств для выплаты займа. Вероятность невозврата в категории "C" равна ~ 8.5%, категории "A" - 8%, категории "B" - 7% и категории "D" - 6%.

### Исследование того, как разные цели кредита влияют на его возврат в срок.

In [33]:
purpose_filtered = data[data['debt'] != 0]
purpose_filtered = purpose_filtered.groupby('purpose_category')['debt'].count()
overall_purpose = data.groupby('purpose_category')['debt'].count()
chance_status = (purpose_filtered / overall_purpose)*100

In [34]:
value = {'clients_overall': overall_purpose,'clients_with_debt': purpose_filtered,'debt_chance': chance_status}
represent = pd.DataFrame(data=value)
display(represent.sort_values(by='debt_chance', ascending=False))

,clients_overall,clients_with_debt,debt_chance
purpose_category,,,
операции с автомобилем,4279,400,9.347978
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803
операции с недвижимостью,10751,780,7.255139


**Вывод:** Большая часть клиентов-должников обозначила "операции с недвижимостью" в качестве цели кредита, поскольку большинство людей берет кредит для покупки квартиры. Меньше всего должников наблюдается среди клиентов, которые взяли кредит на проведение свадьбы. Однако вероятность невозврата выше всего у категории "операции с автомобилем" - 9.3%. Вероятность образования задолженности в категории "получение образования" составляет 9.2%, в категории "проведение свадьбы" ~ 8%, а в категории "операции с недвижимостью - 7.2%.

## Общий вывод.

**Общий вывод:** В ходе исследования был обработан датафрейм с данными о клиентах банка: удалены пропуски и дубликаты, для численных значений выбран наиболее оптимальный формат, категоризированы цели получения кредита. 
Исследование показало, что большая часть задолжников не имеет детей, тем не менее, вероятность не выплатить кредит в срок больше у клиентов с 4 детьми - 9.7%. Также, большинство должников находится в браке, но вероятность невозврата кредита больше всего у клиентов, не состоящих в браке - 9.7%. Больше всего должников входя в категорию "С", но самая высокая вероятность невозврата относится к категории "Е" - 9%. Как правило, большое количество клиентов с задолженностью брали кредит для операций с недвижимостью, а самая высокая вероятность невыплаты кредита относится к операциям с автомобилем - 9.3%.